# Tutorial about the LocData class

In [ ]:
import numpy as np
import pandas as pd

import locan as lc

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

## Sample data

A localization has certain properties such as 'Position_x'. A list of localizations can be assembled into a dataframe:

In [ ]:
df = pd.DataFrame(
    {
        'position_x': np.arange(0,10),
        'position_y': np.random.random(10),
        'frame': np.arange(0,10),
    })

## Instantiate LocData from a dataframe

A LocData object carries localization data together with metadata and aggregated properties for the whole set of localizations.

We first instantiate a LocData object from the dataframe:

In [ ]:
dat = lc.LocData.from_dataframe(dataframe=df)

In [ ]:
attributes = [x for x in dir(dat) if not x.startswith('_')]
attributes

## LocData attributes

The class variable Locdata.count represents the number of all current LocData instantiations.

In [ ]:
print('LocData count: ', lc.LocData.count)

The localization dataset is provided by the data attribute:

In [ ]:
print(dat.data.head())

Aggregated properties are provided by the attribute properties:

In [ ]:
dat.properties

Since spatial coordinates are quite important one can check on *coordinate_labels* and dimension:

In [ ]:
dat.coordinate_labels

In [ ]:
dat.dimension

A numpy array of spatial coordinates is returned by:

In [ ]:
dat.coordinates

## Metadata 

Metadata is provided by the attribute meta and can be printed as

In [ ]:
dat.print_meta()

A summary of the most important metadata is printed as:

In [ ]:
dat.print_summary()

Metadata fields can be printed and changed individually:

In [ ]:
print(dat.meta.comment)
dat.meta.comment = 'user comment'
print(dat.meta.comment)

LocData.meta.map represents a dictionary structure that can be filled by the user. Both key and value have to be strings, if not a TypeError is thrown.

In [ ]:
print(dat.meta.map)
dat.meta.map['user field'] = 'more information'
print(dat.meta.map)

Metadata can also be added at Instantiation:

In [ ]:
dat_2 = lc.LocData.from_dataframe(dataframe=df, meta={'identifier': 'myID_1', 
                                                   'comment': 'my own user comment'})
dat_2.print_summary()

## Instantiate locdata from selection

A LocData object can also be instantiated from a selection of localizations. In this case the LocData object keeps a reference to the original locdata together with a list of indices (or a slice object)). The new dataset is assembled on request of the data attribute.

*Typically a selection is derived using a selection method such that using LocData.from_selection() is not often necessary.*

In [ ]:
dat_2 = lc.LocData.from_selection(dat, indices=[1,2,3,4])
dat_3 = lc.LocData.from_selection(dat, indices=[5,6,7,8])

print('count: ', lc.LocData.count)
print('')
print(dat_2.data)

In [ ]:
dat_2.print_summary()

The reference is kept in a private attribute as are the indices.

In [ ]:
print(dat_2.references)
print(dat_2.indices)

The reference is the same for both selections.

In [ ]:
print(dat_2.references is dat_3.references)

## Instantiate locdata from collection

A LocDat object can further be instantiated from a collection of other LocData objects.

In [ ]:
del(dat_2, dat_3)

dat_1 = lc.LocData.from_selection(dat, indices=[0,1,2])
dat_2 = lc.LocData.from_selection(dat, indices=[3,4,5])
dat_3 = lc.LocData.from_selection(dat, indices=[6,7,8])
dat_c = lc.LocData.from_collection(locdatas=[dat_1, dat_2, dat_3], meta={'identifier': 'my_collection'})

print('count: ', lc.LocData.count, '\n')
print(dat_c.data, '\n')
print(dat_c.properties, '\n')
dat_c.print_summary()

In this case the reference are also kept in case the original localizations from the collected LocData object are requested.

In [ ]:
print(dat_c.references)

In case the collected LocData objects are not needed anymore and should be free for garbage collection the references can be deleted by a dedicated Locdata method

In [ ]:
dat_c.reduce()
print(dat_c.references)

## Concatenating LocData objects 

Lets have a second dataset with localization data:

In [ ]:
del(dat_2)

df_2 = pd.DataFrame(
    {
        'position_x': np.arange(0,10),
        'position_y': np.random.random(10),
        'frame': np.arange(0,10),
    })

dat_2 = lc.LocData.from_dataframe(dataframe=df_2)

print('First locdata:')
print(dat.data.head())
print('')
print('Second locdata:')
print(dat_2.data.head())

In order to combine two sets of localization data into a single LocData object use the class method *LocData.concat*:

In [ ]:
dat_new = lc.LocData.concat([dat, dat_2])
print(f'NUmber of localizations in dat_new: ', len(dat_new))
dat_new.data.head()

## Modifying data in place

In case localization data has been modified in place, i.e. the dataset attribute is changed, all properties and hulls must be recomputed. This is best done by re-instantiating the LocData object using `LocData.from_dataframe()`; but it can also be done using the `LocData.reset()` function.

In [ ]:
del(df, dat)

df = pd.DataFrame(
    {
        'position_x': np.arange(0,10),
        'position_y': np.random.random(10),
        'frame': np.arange(0,10),
    })

dat = lc.LocData.from_dataframe(dataframe=df)

print(dat.data.head())

In [ ]:
dat.centroid

Now if localization data is changed in place (which you should not do unless you have a good reason), properties and bounding box are not automatically adjusted.

In [ ]:
dat.dataframe = pd.DataFrame(
    {
        'position_x': np.arange(0,8),
        'position_y': np.random.random(8),
        'frame': np.arange(0,8),
    })

print(dat.data.head())

In [ ]:
dat.centroid  # so this returns incorrect values here

Update them by re-instantiating a new LocData object:

In [ ]:
new_dat = lc.LocData.from_dataframe(dataframe=dat.data)

In [ ]:
new_dat.centroid

In [ ]:
new_dat.meta

Alternatively you can use `reset()`. In this case, however, metadata is not updated and will provide wrong information.  

In [ ]:
dat.reset()

In [ ]:
dat.centroid

In [ ]:
dat.meta

## Copy LocData

Shallow and deep copies can be made from LocData instances. In either case the class variable count and the metadata is not just copied but adjusted accordingly.

In [ ]:
print('count: ', lc.LocData.count)
print('')
print(dat_2.meta)

In [ ]:
from copy import copy, deepcopy

print('count before: ', lc.LocData.count)
dat_copy = copy(dat_2)
dat_deepcopy = deepcopy(dat_2)
print('count after: ', lc.LocData.count)

In [ ]:
print(dat_copy.meta)

In [ ]:
print(dat_deepcopy.meta)